# 1. Import Dependencies

In [1]:
import cv2 
import uuid
import os
import time

# 2. Define Images Label

In [3]:
COLLECTED_IMG_PATH = r'D:\VisualStudioCode\Rebage-Machine-Learning\Development\Tensorflow\workspace\images\collectedimages'

labels = os.listdir(COLLECTED_IMG_PATH)
labels

['botolkaca',
 'botolplastik',
 'kaleng',
 'kardus',
 'kertas',
 'plastik',
 'sedotan']

# 3. Setup Folders 

In [5]:
IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

In [ ]:
if not os.path.exists(IMAGES_PATH):
    !mkdir {IMAGES_PATH}
        
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        os.mkdir(path)

# 4. Image Labelling

In [ ]:
!pip install --upgrade pyqt5 lxml

In [5]:
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

In [6]:
if not os.path.exists(LABELIMG_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

In [7]:
if os.name == 'posix':
    !make qt5py3
if os.name =='nt':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
# Nanti muncul python gui, pakai python gui itu buat extract fitur
!cd {LABELIMG_PATH} && python labelImg.py

In [11]:
len(os.listdir(os.path.join(COLLECTED_IMG_PATH, 'botolplastik')))

622

# 5. Move them into a Training and Testing Partition

First delete the contents of the train and test directories!!

In [1]:
TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')
ARCHIVE_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'archive.tar.gz')

In [ ]:
if not os.path.exists(TRAIN_PATH):
    os.mkdir(TRAIN_PATH)

if not os.path.exists(TEST_PATH):
    os.mkdir(TEST_PATH)

In [25]:
# Shuffle and copy collected image to train and test dir
import random
import shutil

TRAIN_TO_TOTAL_RATIO = 0.8


for file_folder_name in labels:
    file_dir = os.path.join(IMAGES_PATH, file_folder_name)
    xml_list = [file for file in os.listdir(file_dir) if file.endswith('xml')]
    train_num = int((TRAIN_TO_TOTAL_RATIO*len(xml_list))//1)
    file_for_training = list(random.sample(xml_list, train_num))
    file_for_test = [file for file in xml_list if file not in file_for_training]
    
    for file in file_for_training:
        file_xml = os.path.join(file_dir, file)
        file_jpg = os.path.join(file_dir, file[:-3]) + 'jpg'
        jpg_target = os.path.join(TRAIN_PATH, file[:-3]) + 'jpg'
        xml_target = os.path.join(TRAIN_PATH, file)
        shutil.copyfile(file_xml, xml_target)
        shutil.copyfile(file_jpg, jpg_target)
    
    for file in file_for_test:
        file_xml = os.path.join(file_dir, file)
        file_jpg = os.path.join(file_dir, file[:-3]) + 'jpg'
        jpg_target = os.path.join(TEST_PATH, file[:-3]) + 'jpg'
        xml_target = os.path.join(TEST_PATH, file)
        shutil.copyfile(file_xml, xml_target)
        shutil.copyfile(file_jpg, jpg_target)


In [ ]:
import numpy as np 
import re
import shutil

np.random.seed(101)
# moving random data from classes folder to test and train
test_size = .3
pattern1 = r'(.*)\.[jpg|xml]'

for label_dir in os.listdir(IMAGES_PATH):
    label_dir_path = os.path.join(IMAGES_PATH, label_dir)
    num_files = len( os.listdir( label_dir_path ) )
    print(label_dir, num_files)
    
    if num_files == 0:
        print(f'{label_dir} is copied, continue ...')
        continue
    
    # Karena ada 2 ekstensi file untuk 1 foto (jpg dan xml) 
    # maka diambil set nya
    data_name_temp = []
    for data in os.listdir(label_dir_path):
        searched = re.search(pattern1, data)
        data_name_temp.append(searched.groups()[0])
    
    data_name_temp = np.array(list(set(data_name_temp)), dtype=np.str)
    
    # untuk setiap data akan diacak 
    np.random.shuffle(data_name_temp)
    
    # Todo: Melakukan pengambilan data 
    break_point = int(len(data_name_temp) * test_size)
    
    test_candidate = data_name_temp[:break_point]
    
    # untuk setiap kandidat data test akan dipindahkan ke folder test 
    for data_test in test_candidate:
        # melakukan pencarian tiap data test pada direktori label dir path
        pattern2 = r'(' + data_test + r'\.[xmljpg]*)'
        
        for train_test_data in os.listdir(label_dir_path):
            searched = re.search(pattern2, train_test_data)
            # kalau bukan none maka test
            if searched is not None:
                test_data_use = searched.groups()[0]
                source_path = os.path.join(label_dir_path, test_data_use)
                # print("Test : ", source_path)
                shutil.copy(source_path, TEST_PATH)
            # jika none maka train
            else:
                source_path = os.path.join(label_dir_path, train_test_data)
                # print("Train : ", source_path)
                shutil.copy(source_path, TRAIN_PATH)
    # break

Cek ukuran test dan train

In [ ]:
print(f"Train data : {len(os.listdir(TRAIN_PATH))} data")
print(f"Test data : {len(os.listdir(TEST_PATH))} data")

# (OPSIONAL) 7. File tar jika mau menyimpan data dalam bentuk TAR dan diakses dari Google Collab

In [ ]:
import tarfile

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        for source in source_dir:
            tar.add(source, arcname=os.path.basename(source))

In [ ]:
make_tarfile(ARCHIVE_PATH, [TRAIN_PATH, TEST_PATH])